拼多多2019笔试题。给若干任务，每个任务有完成时间与依赖。现限制单线程工作，求一个使得平均响应时间最小的完成方案。

思路：拓扑排序的变种。

In [ ]:
import sys

n, m = map(int, sys.stdin.readline().strip().split())
d_arr = [0] * n  # 入度数组
adj_table = {job_idx: list() for job_idx in range(n)}
ts = list(map(int, sys.stdin.readline().strip().split()))
for _ in range(m):
    # from, to
    f, t = map(int, sys.stdin.readline().strip().split())
    if f - 1 not in adj_table:
        adj_table[f - 1] = list()
    adj_table[f - 1].append(t - 1)
    d_arr[t - 1] += 1


def func(n, d_arr, ts, adj_table):
    q = list()
    t = 0
    res = list()

    # 找到第一个入度为0且所需时间最少的任务
    cur_min = 0x7FFFFFFF
    cur_job = -1
    for job_idx, d_in in enumerate(d_arr):
        if d_in == 0 and ts[job_idx] < cur_min:
            cur_job, cur_min = job_idx, ts[job_idx]
    q.append(cur_job)

    # 完成n个任务
    for epoch in range(n):
        cur_job = q.pop(0)
        d_arr[cur_job] -= 1
        res.append(cur_job+1)

        if epoch < n:
            for post_job in adj_table[cur_job]:
                d_arr[post_job] -= 1

            bak_job = -1
            cur_min = 0x7FFFFFFF
            for job_idx, d_in in enumerate(d_arr):
                if d_in == 0 and ts[job_idx] < cur_min:
                    bak_job, cur_min = job_idx, ts[job_idx]
            q.append(bak_job)

    return res

[Network Delay Time](https://leetcode.com/problems/network-delay-time/)。给一个有$N$个节点的有向图，再给出一若干节点之间的有向距离```times```。现从节点$K$发出一条消息，问经过多少时间后网络中的所有节点均已收到消息。索引从$1$开始。

思路：BFS。首先维护一个长度$N$的数组来表示各节点收到消息的时间，用大值初始化，其中```arr[K]=0```。然后开始BFS，所有节点只保留收到消息的最小时间。难点在于如何避免重复访问，该题数据是有环的。设置访问数组是错误的，会漏边；正确的做法是**只将那些更新过时间的邻居节点加入队列**。

In [2]:
def networkDelayTime(times, N: int, K: int) -> int:
    adj_table = {idx: list() for idx in range(1, N+1)}
    for f, t, time in times:
        adj_table[f].append((t, time))

    res = [0x7FFFFFFF]*(N+1)
    res[K] = 0
    q = [(K, 0)]    # (node_idx,cur_time)

    while q:
        vis_node, cur_time = q.pop(0)

        for t, time in adj_table[vis_node]:
            if cur_time+time < res[t]:    # 只有当时间更新了才加入队列，这是避免重复访问的关键
                res[t] = cur_time+time
                q.append((t, cur_time+time))

    return max(res[1:]) if max(res[1:]) < 0x7FFFFFFF else -1

2

[Possible Bipartition](https://leetcode.com/problems/possible-bipartition/)。染色问题，给$N$个点，然后给出这些点的连接关系，现有两种颜色，直接相连的点不能同色。判断能否存在染色方案。

思路：BFS。因为只有两种颜色，首先随便选一个点进行染色，那么与它相邻的所有点均只能染另外一种颜色。设置一个颜色数组，记录每一个点的两种颜色，同时还起到一个访问数组的作用。

In [3]:
def possibleBipartition(N: int, dislikes) -> bool:
    adj_table = {idx: list() for idx in range(N)}
    for item in dislikes:
        i, j = item[0]-1, item[1]-1
        adj_table[i].append(j)
        adj_table[j].append(i)

    colors = [None]*N
    for idx in range(N):
        if colors[idx] is None:
            q = [idx]
            colors[idx] = True
            while q:
                vis_node = q.pop(0)
                for neighbor in adj_table[vis_node]:
                    if colors[neighbor] is None:    # 未被染色，染色并入队
                        colors[neighbor] = not colors[vis_node]
                        q.append(neighbor)
                    elif colors[neighbor] == colors[vis_node]:    # 冲突返回false
                        return False
                    else:
                        continue

    return True

True

[Is Graph Bipartite?](https://leetcode.com/problems/is-graph-bipartite/)。给一图，判断该图是否是二分图。二分图即把该图的所有节点分到两个集合中，每个集合中的任意两个节点都不存在边。

思路：染色问题。即判断所有直接相连的点能否染成不同的颜色。维护一个染色数组，初始化为None，然后对图做BFS。因为图可能存在孤岛，所以当队列为空时，需要随即取出一个未染色的点加入队列。

In [7]:
def isBipartite(graph) -> bool:
    n = len(graph)
    color_arr = [None]*n
    color_arr[0] = True
    q = [0]

    while q:
        vis_node = q.pop(0)
        for neighbor in graph[vis_node]:
            if color_arr[neighbor] == color_arr[vis_node]:
                return False
            else:
                if color_arr[neighbor] is None:
                    color_arr[neighbor] = not color_arr[vis_node]
                    q.append(neighbor)

        if not q:    # 当出现孤岛时
            for idx in range(n):
                if color_arr[idx] is None:
                    color_arr[idx] = True
                    q.append(idx)
                    break

    return True

False

[Find Eventual Safe States](https://leetcode.com/problems/find-eventual-safe-states/)。以数组的形式给一有向图，要求所有满足以下性质的节点：从该节点出发，无论怎么走都会到达一个最终节点。

思路：该题其实就是逆拓扑排序，首先将整个图反向，然后开始进行拓扑排序，能排序的点即是所求。

In [8]:
def eventualSafeNodes(graph):
    n = len(graph)
    adj_table = {idx: list() for idx in range(n)}
    D_in = [0]*n

    for idx, l in enumerate(graph):
        D_in[idx] = len(l)
        for i in l:
            adj_table[i].append(idx)

    q = list()
    for idx in range(n):
        if D_in[idx] == 0:
            q.append(idx)

    res = list()
    while q:
        vis_node = q.pop(0)
        res.append(vis_node)
        D_in[vis_node] -= 1
        for neighbor in adj_table[vis_node]:
            D_in[neighbor] -= 1
            if D_in[neighbor] == 0:
                q.append(neighbor)

    return sorted(res)

[5, 6, 2, 4]

[All Nodes Distance K in Binary Tree](https://leetcode.com/problems/all-nodes-distance-k-in-binary-tree/)。给一二叉树与一个目标节点```target```，在二叉树中找到所有跟```target```距离为$K$的节点。

思路：将二叉树转成无向图就好做了，直接以```target```为起点做DFS即可。

In [ ]:
def distanceK(root, target, K):
    graph = dict()

    def build_G(graph, root):
        if not root:
            return
        graph.setdefault(root, list())
        if root.left:
            graph.setdefault(root.left, list())
            graph[root].append(root.left)
            graph[root.left].append(root)
            build_G(graph, root.left)
        if root.right:
            graph.setdefault(root.right, list())
            graph[root].append(root.right)
            graph[root.right].append(root)
            build_G(graph, root.right)

    build_G(graph, root)
    res = list()
    visited = {ptr: False for ptr in list(graph.keys())}

    def dfs(idx, path):
        if path == K:
            res.append(idx.val)
            return

        for neighbor in graph[idx]:
            if not visited[neighbor]:
                visited[idx] = True
                dfs(neighbor, path+1)
                visited[idx] = False

    dfs(target, 0)
    return res

[二分图最大匹配](https://uoj.ac/problem/78)。给一二分图，有A、B两种节点，给出最大匹配的数量及匹配方案。

思路：匈牙利算法求解最大匹配。

In [ ]:
import sys


def data_load():
    n_u, n_v, n_row = map(int, sys.stdin.readline().strip().split())
    adj_tab = {node_idx: list() for node_idx in range(n_v)}  # 仅记录B类型节点的邻接表即可

    for _ in range(n_row):
        u, v = map(int, sys.stdin.readline().strip().split())
        adj_tab[v - 1].append(u - 1)  # 输入下标从1开始，改成0

    return n_u, n_v, adj_tab


def find_aug_path_dfs(cur_node: int, alter_flag: list, adj_tab: dict) -> bool:
    '''
    dfs寻找增广路径
    :param cur_node: 当前节点，只会根据交替路径在单边节点中移动
    :param alter_flag: 交替路径访问标记
    :param adj_tab: 邻接表
    :return:
    '''
    for neighbor in adj_tab[cur_node]:  # 遍历所有邻居
        if not alter_flag[neighbor]:  # 避免环路
            alter_flag[neighbor] = 1
            # 利用短路特性，若找到一个未匹配的邻居，则说明找到一条更长的增广路径
            # 若邻居已被匹配，则直接跳过邻居移动到邻居的匹配节点，保证了cur_node的类型不变
            # unmatch -> match -> match -> unmatch，cur_node会从位置1跳到位置3
            if (matchs[neighbor] < 0) or (find_aug_path_dfs(matchs[neighbor], alter_flag, adj_tab)):
                matchs[neighbor] = cur_node    # 在记录匹配时，邻居(后一个节点)作为键值
                return True

    return False


if __name__ == "__main__":
    n_a, n_b, adj_tab = data_load()

    matchs = [-1] * n_a  # 每个a匹配的b

    res = 0
    for i in range(n_b):  # 遍历图中B类型的所有点
        alter_flag = [0] * n_a  # 交替路径访问标记，仅需要记录A节点的访问情况即可
        if find_aug_path_dfs(i, alter_flag, adj_tab):
            res += 1

    print(res)
    for a in range(n_a):
        print(matchs[a] + 1, end=' ')  # 下标从0开始转成从1开始